# Utility Chains Overview

## Summarizing Documents

### load_summarize_chain

In [ ]:
#Please install the package as per your requirement :)
#!pip install openai==1.14.2
#!pip install langchain==0.1.13
#!pip install huggingface-hub==0.21.4
#!pip install langchain-openai==0.1.0
#!pip install tiktoken==0.5.2
#!pip install bs4==0.0.2

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "sk-JqqHQBChzONQOeXSw_7mRSUQDMmloJwzgiPMK9JQeVT3BlbkFJ7m3JDlCQ1xtIAp_8lFF7H3sL3NlxtgZjEPz-8Hh0kA"

In [ ]:
#The below import has been replaced by the later one
from langchain.llms import OpenAI
# from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import CharacterTextSplitter
from langchain.docstore.document import Document

In [ ]:
llm = OpenAI(temperature=0.9)

In [6]:
# Reading the document
with open("sample.txt") as f:
    data = f.read()

<font color='green'>
When it comes to document processing, breaking a large document into smaller, more manageable chunks is essential
<font>

In [8]:
# Split text
text_splitter = CharacterTextSplitter()
texts = text_splitter.split_text(data)

In [9]:
# Create multiple documents
docs = [Document(page_content=t) for t in texts]

In [10]:
docs

[Document(metadata={}, page_content="The Computer: Revolutionizing the World of Technology\\n\\nIntroduction:\\nThe computer is a marvel of human ingenuity that has revolutionized the world in countless ways. From its humble beginnings as a complex calculating machine to its current status as a ubiquitous tool in every aspect of our lives, the computer has transformed how we work, communicate, learn, and entertain ourselves. This essay explores the evolution, impact, and future potential of computers in shaping our modern world.\\n\\nThe Birth of the Computer:\\nThe computer has its roots in the early 19th century when inventors and mathematicians began conceptualizing machines capable of automating complex calculations. However, it was not until the mid-20th century that the first electronic digital computers were developed. Pioneers such as Alan Turing, John von Neumann, and Grace Hopper made significant contributions to the field, laying the groundwork for the computers we know toda

<font color='green'>
To create an instance of load_summarizer_chain, we need to provide three arguments. <br><br>Firstly, we need to pass the desired large language model that will be used to query the user input. Secondly, we specify the type of langchain chain to be used for summarizing documents.<br> Lastly, we can set the verbose argument to True if we want to see all the intermediate steps involved in processing the user request and generating the output.<font>

In [11]:
chain = load_summarize_chain(llm, chain_type="map_reduce", verbose=True)
chain.invoke(docs)



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"The Computer: Revolutionizing the World of Technology\n\nIntroduction:\nThe computer is a marvel of human ingenuity that has revolutionized the world in countless ways. From its humble beginnings as a complex calculating machine to its current status as a ubiquitous tool in every aspect of our lives, the computer has transformed how we work, communicate, learn, and entertain ourselves. This essay explores the evolution, impact, and future potential of computers in shaping our modern world.\n\nThe Birth of the Computer:\nThe computer has its roots in the early 19th century when inventors and mathematicians began conceptualizing machines capable of automating complex calculations. However, it was not until the mid-20th century that the first electronic digital computers were developed. Pioneers such as Alan Turing, John von Neumann, an

{'input_documents': [Document(metadata={}, page_content="The Computer: Revolutionizing the World of Technology\\n\\nIntroduction:\\nThe computer is a marvel of human ingenuity that has revolutionized the world in countless ways. From its humble beginnings as a complex calculating machine to its current status as a ubiquitous tool in every aspect of our lives, the computer has transformed how we work, communicate, learn, and entertain ourselves. This essay explores the evolution, impact, and future potential of computers in shaping our modern world.\\n\\nThe Birth of the Computer:\\nThe computer has its roots in the early 19th century when inventors and mathematicians began conceptualizing machines capable of automating complex calculations. However, it was not until the mid-20th century that the first electronic digital computers were developed. Pioneers such as Alan Turing, John von Neumann, and Grace Hopper made significant contributions to the field, laying the groundwork for the co

## HTTP Requests

### LLMRequestsChain

In [12]:
from langchain.chains import LLMRequestsChain, LLMChain

In [13]:
template = """
Extract the answer to the question '{query}' or say "not found" if the information is not available.
{requests_result}
"""

PROMPT = PromptTemplate(
    input_variables=["query", "requests_result"],
    template=template,
)

In [14]:
llm=OpenAI()

In [15]:
chain = LLMRequestsChain(llm_chain=LLMChain(llm=llm, prompt=PROMPT))

C:\Users\002M3W744\AppData\Local\Temp\ipykernel_26876\2809102635.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMRequestsChain(llm_chain=LLMChain(llm=llm, prompt=PROMPT))


<font color='green'>
Preparing the question & inputs to the http request<font>

In [16]:
question = "What is the capital of india?"
inputs = {
    "query": question,
    "url": "https://www.google.com/search?q=" + question.replace(" ", "+"),
}

In [17]:
chain.invoke(inputs)

{'query': 'What is the capital of india?',
 'url': 'https://www.google.com/search?q=What+is+the+capital+of+india?',
 'output': '\nNew Delhi is the capital of India. '}

<font color='green'>
Let's look at the internal functioning<font>

In [18]:
import inspect
print(inspect.getsource(chain._call))

    def _call(
        self,
        inputs: Dict[str, Any],
        run_manager: Optional[CallbackManagerForChainRun] = None,
    ) -> Dict[str, Any]:
        from bs4 import BeautifulSoup

        _run_manager = run_manager or CallbackManagerForChainRun.get_noop_manager()
        # Other keys are assumed to be needed for LLM prediction
        other_keys = {k: v for k, v in inputs.items() if k != self.input_key}
        url = inputs[self.input_key]
        res = self.requests_wrapper.get(url)
        # extract the text from the html
        soup = BeautifulSoup(res, "html.parser")
        other_keys[self.requests_key] = soup.get_text()[: self.text_length]
        result = self.llm_chain.predict(  # type: ignore[attr-defined]
            callbacks=_run_manager.get_child(), **other_keys
        )
        return {self.output_key: result}

